In [1]:
from collections import namedtuple
import os
os.chdir("..")
os.getcwd()


'd:\\ineuron\\DL_project\\CNN_project'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path       #base untrained vgg16 archtecture model
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    
  

In [8]:
from cnnProject.constant import *
from cnnProject.utils import read_yaml,create_directories

In [9]:
class ConfigrationManger:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])

    def get_training_config(self) -> TrainingConfig :
        training = self.config.training
        prepare_base_model = self.config.base_model_preparation
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"PetImages")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs =  params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTAION,
            params_image_size = params.IMAGE_SIZE
            )

        return training_config


In [ ]:
#training Component
import os
import time
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class Training:
    def __init__(self,config:TrainingConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        #will be doing augmentation 
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],  #only takes rows and columns, channels not aka index 0,1
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear",
            # interpolation is a process of determining the unknown values that lie in between the known data points.
            # ex find value between 1 and 2 on a line. 
        )


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.trained_model_path,
            subset= "validation",
            shuffle=False,
            **dataflow_kwargs
        )

    


In [ ]:
try:
    config = ConfigrationManger()
    callback_preparation_config = config.get_callback_perparation_config()
    callback_prepare = CallbackPreparation(config=callback_preparation_config)
    callback_lst = callback_prepare.get_tb_checkpoint_callback()



    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()  #create a training and validation generator similar to train_test_split but not exactly  same 
    training.train(
        callback_list = callback_lst,
    )
except Exception as e:
    raise e